In [1]:
import numpy as np
import cv2
import os
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# Define the data directory
data_dir = pathlib.Path(".\\dataset")

In [3]:
# Define the labels and label dictionary
labels = ['dwaynejohnson', 'emmawatson', 'johnnydepp', 'ladygaga', 'leonardodicaprio']
label_dict = {label: i for i, label in enumerate(labels)}


In [4]:
# Load and preprocess data
X, y = [], []

for label in labels:
    images = list(data_dir.glob(f'{label}/*.jpg'))
    for image in images:
        img = cv2.imread(str(image))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        img = cv2.resize(img, (224, 224))  # Resize to a common size
        X.append(img)
        y.append(label_dict[label])

X = np.array(X) / 255.0
y = np.array(y)

In [5]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Define a pre-trained model (e.g., MobileNetV2) for transfer learning
base_model = keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model layers


In [7]:
# Build the custom classification head
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(labels), activation='softmax')
])



In [8]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [9]:
# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))



Epoch 1/10
8/8 [==============================] - 17s 1s/step - loss: 1.1747 - accuracy: 0.5932 - val_loss: 0.9765 - val_accuracy: 0.6441
Epoch 2/10
8/8 [==============================] - 8s 1s/step - loss: 0.4478 - accuracy: 0.8559 - val_loss: 0.8725 - val_accuracy: 0.6780
Epoch 3/10
8/8 [==============================] - 8s 1s/step - loss: 0.2541 - accuracy: 0.9364 - val_loss: 0.5684 - val_accuracy: 0.7797
Epoch 4/10
8/8 [==============================] - 8s 988ms/step - loss: 0.1483 - accuracy: 0.9576 - val_loss: 0.5228 - val_accuracy: 0.7797
Epoch 5/10
8/8 [==============================] - 8s 981ms/step - loss: 0.1003 - accuracy: 0.9788 - val_loss: 0.6389 - val_accuracy: 0.8136
Epoch 6/10
8/8 [==============================] - 8s 988ms/step - loss: 0.0593 - accuracy: 0.9958 - val_loss: 0.4463 - val_accuracy: 0.8136
Epoch 7/10
8/8 [==============================] - 8s 981ms/step - loss: 0.0406 - accuracy: 0.9958 - val_loss: 0.4311 - val_accuracy: 0.8136
Epoch 8/10
8/8 [============

In [10]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")



2/2 [==============================] - 2s 717ms/step - loss: 0.4726 - accuracy: 0.7966
Test accuracy: 0.7966101765632629


In [11]:
# Save the trained model
model.save('model.keras')